# Functions and Imports - no pyiron dependence

In [1]:
from h5io_browser.base import read_nested_dict_from_hdf, list_hdf, write_dict_to_hdf
import os

In [2]:
from dataclasses import dataclass
import numpy as np
from pint import UnitRegistry
from typing import Optional, List

In [3]:
@dataclass
class OutputGenericDFT:
    energy_free: np.ndarray
    n_valence: dict
    bands_k_weights: np.ndarray
    kpoints_cartesian: np.ndarray
    bands_e_fermi: np.ndarray
    bands_occ: np.ndarray
    bands_eigen_values: np.ndarray
    scf_convergence: List[bool]
    scf_energy_int: np.ndarray
    scf_energy_free: np.ndarray
    scf_computation_time: np.ndarray
    scf_energy_zero: np.ndarray
    scf_energy_band: np.ndarray
    scf_electronic_entropy: np.ndarray
    scf_residue: np.ndarray
    energy_int: np.ndarray
    computation_time: np.ndarray
    energy_zero: np.ndarray
    energy_band: np.ndarray
    electronic_entropy: np.ndarray
    residue: np.ndarray

@dataclass
class GenericOutput:
    cells: np.ndarray  # N_steps * 3 *3  [Angstrom]
    energy_pot: np.ndarray  # N_steps  [eV]
    energy_tot: np.ndarray  # N_steps  [eV]
    forces: np.ndarray  # N_steps * N_atoms * 3  [eV/Angstrom]
    indices: Optional[np.ndarray]  # N_steps * N_atoms
    natoms: Optional[np.ndarray]  # N_steps
    positions: np.ndarray  # N_steps * N_atoms * 3  [Angstrom]
    pressures: Optional[np.ndarray]  # N_steps * 3 * 3
    steps: Optional[np.ndarray]  # N_steps
    temperature: Optional[np.ndarray]  # N_steps
    unwrapped_positions: Optional[np.ndarray]  # N_steps * N_atoms * 3  [Angstrom]
    velocities: Optional[np.ndarray]  # N_steps * N_atoms * 3  [Angstrom/fs]
    volume: np.ndarray  # N_steps
    dft: Optional[OutputGenericDFT]

@dataclass
class Server:
    user: str
    host: str
    run_mode: str
    queue: Optional[str]
    qid: Optional[int]
    cores: int
    threads: int
    new_h5: bool
    structure_id: Optional[int]
    run_time: int  # [seconds]
    memory_limit: Optional[str]
    accept_crash: bool

@dataclass
class Executable:
    version: str
    name: str
    operation_system_nt: bool
    executable: Optional[str]
    mpi: bool
    accepted_return_codes: List[int]

@dataclass
class GenericDict:
    restart_file_list: list
    restart_file_dict: dict
    exclude_nodes_hdf: list
    exclude_groups_hdf: list

@dataclass
class Interactive:
    interactive_flush_frequency: int
    interactive_write_frequency: int

@dataclass
class GenericInput:
    calc_mode: str
    structure: str
    fix_symmetry: Optional[bool]
    k_mesh_spacing: Optional[float]
    k_mesh_center_shift: Optional[np.ndarray]
    reduce_kpoint_symmetry: Optional[bool]
    restart_for_band_structure: Optional[bool]
    path_name: Optional[str]
    n_path: Optional[str]
    fix_spin_constraint: Optional[bool]
    max_iter: Optional[int]
    temperature: Optional[float]
    n_ionic_steps: Optional[int]
    n_print: Optional[int]
    temperature_damping_timescale: Optional[float]
    pressure_damping_timescale: Optional[float]
    time_step: Optional[int]

@dataclass
class LammpsPotential:
    citation: Optional[str]
    config: List[str]
    filename: List[str]
    model: str
    name: str
    species: List[str]

@dataclass
class Units:
    length: str
    mass: str

@dataclass
class Cell:
    cell: np.ndarray  # 3 * 3   [Angstrom]
    pbc: np.ndarray  # 3

@dataclass
class Structure:
    dimension: int
    indices: np.array
    info: dict
    positions: np.ndarray  # N_atoms * 3  [Angstrom]
    species: List[str]
    cell: Cell
    units: Units

@dataclass
class LammpsInputFiles:
    control_inp: str
    potential_inp: str

@dataclass
class LammpsInput:
    generic_dict: GenericDict
    interactive: Interactive
    generic: GenericInput
    structure: Structure
    potential: LammpsPotential
    input_files: LammpsInputFiles

@dataclass 
class LammpsOutput:
    generic: GenericOutput

@dataclass
class LammpsJob:
    calculation_input: LammpsInput
    executable: Executable
    server: Server
    calculation_output: LammpsOutput

@dataclass
class Species:
    name: str
    pot_type: str
    element: str
    potential: str

@dataclass
class PawPot:
    species: List[Species]

@dataclass
class SphinxAtom:
    label: str
    coords: np.ndarray  # 3  [Angstrom]
    movable: bool

@dataclass
class SphinxElement:
    element: str
    atom: List[SphinxAtom]

@dataclass
class SphinxStructure:
    cell: np.ndarray  # 3 * 3  [Angstrom]
    species: List[SphinxElement]

@dataclass
class SphinxKpoint:
    coords: np.ndarray  # 3 
    weight: float
    relative: bool

@dataclass
class SphinxBasis:
    e_cut: float  # [Hartree]
    k_point: SphinxKpoint
    folding: np.ndarray  # 3
    save_memory: bool

@dataclass
class SphinxPreConditioner:
    type: str
    scaling: float
    spin_scaling: float

@dataclass
class ScfDiag:
    rho_mixing: float
    spin_mixing: float
    delta_energy: float
    max_steps: int
    preconditioner: SphinxPreConditioner
    block_ccg: dict

@dataclass
class SphinxPawHamiltonian:
    number_empty_states: int
    ekt: float
    methfessel_paxton: bool
    xc: str
    spin_polarized: bool

@dataclass
class SphinxWaves:
    paw_basis: bool
    lcao: dict

@dataclass
class SphinxRho:
    atomic_orbitals: int

@dataclass
class SphinxInitialGuess:
    waves: SphinxWaves
    rho: SphinxRho
    no_waves_storage: bool

@dataclass
class BornOppenheimer:
    scf_diag: ScfDiag

@dataclass
class SphinxRicQN:
    max_steps: int
    max_step_length: float
    born_oppenheimer: BornOppenheimer

@dataclass
class SphinxMain:
    ric_qn: SphinxRicQN

@dataclass
class SphinxInternalInput:
    paw_pot: PawPot
    structure: SphinxStructure
    basis: SphinxBasis
    paw_hamilton: SphinxPawHamiltonian
    initial_guess: SphinxInitialGuess
    main: SphinxMain

@dataclass
class SphinxInputParameters:
    sphinx: SphinxInternalInput
    encut: float
    kpointcoords: List[float]  # 3
    kpointfolding: List[int]  # 3
    empty_states: int
    methfessel_paxton: bool
    sigma: float 
    xcorr: str
    vasppot: bool
    e_step: int
    ediff: float
    write_waves: bool
    kj_xc: bool
    save_memory: bool
    rho_mixing: float
    spin_mixing: float
    rho_residual_scaling: float
    spin_residual_scaling: float
    check_overlap: bool
    threads: bool
    use_on_the_fly_cg_optimization: bool
    ionic_step: int

@dataclass
class SphinxOutputChargeDensity:
    total: np.ndarray

@dataclass
class SphinxDensityOfStates:
    energies: str
    int_densities: str
    tot_densities: str

@dataclass
class SphinxElectronicStructure:
    efermi: float
    eig_matrix: np.ndarray
    k_points: np.ndarray
    k_weights: np.ndarray
    occ_matrix: np.ndarray
    dos: SphinxDensityOfStates

@dataclass
class SphinxElectrostaticPotential:
    total: np.ndarray

@dataclass
class SphinxOutput:
    charge_density: SphinxOutputChargeDensity
    electronic_structure: SphinxElectronicStructure
    electrostatic_potential: SphinxElectrostaticPotential
    generic: GenericOutput

@dataclass
class SphinxInput:
    generic_dict: GenericDict
    interactive: Interactive
    generic: GenericInput
    parameters: SphinxInputParameters
    structure: Structure

@dataclass
class SphinxJob:
    executable: Executable
    server: Server
    calculation_input: SphinxInput
    calculation_output: SphinxOutput
    job_id: int
    status: str

In [4]:
def get_node_from_job_dict(job_dict, node):
    node_name_lst = node.split("/")
    tmp_dict = job_dict
    for group in node_name_lst:
        tmp_dict = tmp_dict[group]
    return tmp_dict

In [5]:
def convert_generic_parameters_to_string(generic_parameter_dict):
    output_str = ""
    for p, v in zip(generic_parameter_dict["data_dict"]["Parameter"], generic_parameter_dict["data_dict"]["Value"]):
        output_str += p.replace("___", " ") + " " + v + "\n"
    return output_str[:-1]

In [6]:
def convert_generic_parameters_to_dictionary(generic_parameter_dict):
    return {p: v for p, v in zip(generic_parameter_dict["data_dict"]["Parameter"], generic_parameter_dict["data_dict"]["Value"])}

In [7]:
def filter_dict(input_dict, remove_keys_lst):
    def recursive_filter(input_value, remove_keys_lst):
        if isinstance(input_value, dict):
            return filter_dict(input_dict=input_value, remove_keys_lst=remove_keys_lst)
        else:
            return input_value 
    
    return {k: recursive_filter(input_value=v, remove_keys_lst=remove_keys_lst) for k, v in input_dict.items() if k not in remove_keys_lst}

In [8]:
def sort_dictionary_from_datacontainer(input_dict):
    def recursive_sort(input_value):
        if isinstance(input_value, dict):
            return sort_dictionary_from_datacontainer(input_dict=input_value)
        else:
            return input_value 
    
    ind_dict, content_dict = {}, {}
    content_lst_flag = False
    for k, v in input_dict.items():
        if "__index_" in k:
            key, ind = k.split("__index_")
            if key ==  "":
                content_lst_flag = True
                ind_dict[int(ind)] = recursive_sort(input_value=v)               
            else:
                ind_dict[int(ind)] = key
                content_dict[key] = recursive_sort(input_value=v)
        else:
            content_dict[k] = recursive_sort(input_value=v)
    if content_lst_flag:
        return [ind_dict[ind] for ind in sorted(list(ind_dict.keys()))]
    elif len(ind_dict) == len(content_dict):
        return {ind_dict[ind]: content_dict[ind_dict[ind]] for ind in sorted(list(ind_dict.keys()))}
    elif len(ind_dict) == 0: 
        return content_dict
    else:
        raise KeyError()

In [9]:
def convert_datacontainer_to_dictionary(data_container_dict):
    return sort_dictionary_from_datacontainer(input_dict=filter_dict(
        input_dict=data_container_dict, 
        remove_keys_lst=["NAME", "TYPE", "OBJECT", "HDF_VERSION", "READ_ONLY", "VERSION"],
    ))

# Create SPHInX job

In [10]:
from pyiron_atomistics import Project

In [11]:
pr = Project("test")

In [12]:
pr.remove_jobs(recursive=True, silently=True)

0it [00:00, ?it/s]

In [13]:
structure = pr.create.structure.ase.bulk("Al", cubic=True)

In [14]:
job = pr.create.job.Sphinx("sx")

In [15]:
job.structure =structure

In [16]:
job.calc_minimize()

In [17]:
job.run()

The job sx was saved and received the ID: 1


# Read job dictionary

In [18]:
job_dict = read_nested_dict_from_hdf(
    file_name=job.project_hdf5.file_name,
    h5_path="/",
    recursive=True,
    slash='ignore',
)

In [19]:
sphinx_input_parameter_dict = convert_datacontainer_to_dictionary(data_container_dict=job_dict["sx"]["input"]["parameters"])

In [20]:
generic_input_dict = convert_generic_parameters_to_dictionary(
    generic_parameter_dict=job_dict["sx"]["input"]['generic'],
)

In [21]:
output_dict = convert_datacontainer_to_dictionary(data_container_dict=job_dict["sx"]["output"]["generic"])

In [22]:
ureg = UnitRegistry()

In [23]:
sphinx_job = SphinxJob(
    executable=Executable(
        version=job_dict["sx"]["executable"]["executable"]["version"],
        name=job_dict["sx"]["executable"]["executable"]["name"],
        operation_system_nt=job_dict["sx"]["executable"]["executable"]["operation_system_nt"],
        executable=job_dict["sx"]["executable"]["executable"]["executable"],
        mpi=job_dict["sx"]["executable"]["executable"]["mpi"],
        accepted_return_codes=job_dict["sx"]["executable"]["executable"]["accepted_return_codes"],
    ),
    server=Server(
        user=job_dict["sx"]["server"]["user"],
        host=job_dict["sx"]["server"]["host"],
        run_mode=job_dict["sx"]["server"]["run_mode"],
        queue=job_dict["sx"]["server"]["queue"],
        qid=job_dict["sx"]["server"]["qid"],
        cores=job_dict["sx"]["server"]["cores"],
        threads=job_dict["sx"]["server"]["threads"],
        new_h5=job_dict["sx"]["server"]["new_h5"],
        structure_id=job_dict["sx"]["server"]["structure_id"],
        run_time=job_dict["sx"]["server"]["run_time"],
        memory_limit=job_dict["sx"]["server"]["memory_limit"],
        accept_crash=job_dict["sx"]["server"]["accept_crash"],
    ),
    calculation_input=SphinxInput(
        generic_dict=GenericDict(
            restart_file_list=job_dict["sx"]["input"]['generic_dict']['restart_file_list'],
            restart_file_dict=job_dict["sx"]["input"]['generic_dict']['restart_file_dict'],
            exclude_nodes_hdf=job_dict["sx"]["input"]['generic_dict']['exclude_nodes_hdf'],
            exclude_groups_hdf=job_dict["sx"]["input"]['generic_dict']['exclude_groups_hdf'],
        ),
        interactive=Interactive(
            interactive_flush_frequency=job_dict["sx"]["input"]['interactive']["interactive_flush_frequency"],
            interactive_write_frequency=job_dict["sx"]["input"]['interactive']["interactive_flush_frequency"],
        ),
        generic=GenericInput(
            calc_mode=generic_input_dict["calc_mode"],
            structure=generic_input_dict["structure"],
            fix_symmetry=generic_input_dict.get("fix_symmetry", None),
            k_mesh_spacing=generic_input_dict.get("k_mesh_spacing", None),
            k_mesh_center_shift=generic_input_dict.get("k_mesh_center_shift", None),
            reduce_kpoint_symmetry=generic_input_dict.get("reduce_kpoint_symmetry", None),
            restart_for_band_structure=generic_input_dict.get("restart_for_band_structure", None),
            path_name=generic_input_dict.get("path_name", None),
            n_path=generic_input_dict.get("n_path", None),
            fix_spin_constraint=generic_input_dict.get("fix_spin_constraint", None),
            max_iter=generic_input_dict.get("max_iter", None),
            temperature=generic_input_dict.get("temperature", None),
            n_ionic_steps=generic_input_dict.get("n_ionic_steps", None),
            n_print=generic_input_dict.get("n_print", None),
            temperature_damping_timescale=generic_input_dict.get("temperature_damping_timescale", None),
            pressure_damping_timescale=generic_input_dict.get("pressure_damping_timescale", None),
            time_step=generic_input_dict.get("time_step", None),
        ),
        parameters=SphinxInputParameters(
            sphinx=SphinxInternalInput(
                paw_pot=PawPot(
                    species=[
                        Species(name=s["name"], pot_type=s["potType"], element=s["element"], potential=s["potential"]) 
                        for s in sphinx_input_parameter_dict['sphinx']["pawPot"]["species"]
                    ]
                ),
                structure=SphinxStructure(
                    cell=sphinx_input_parameter_dict['sphinx']["structure"]["cell"],
                    species=[
                        SphinxElement(
                            element=s["element"],
                            atom=[
                                SphinxAtom(label=a["label"], coords=a["coords"], movable=bool(a["movable"])) 
                                for a in s["atom"]
                            ],
                        )
                        for s in sphinx_input_parameter_dict['sphinx']["structure"]["species"]
                    ]
                ),
                basis=SphinxBasis(
                    e_cut=sphinx_input_parameter_dict['sphinx']["basis"]["eCut"],
                    k_point=SphinxKpoint(
                        coords=sphinx_input_parameter_dict['sphinx']["basis"]["kPoint"]["coords"], 
                        weight=sphinx_input_parameter_dict['sphinx']["basis"]["kPoint"]["weight"], 
                        relative=bool(sphinx_input_parameter_dict['sphinx']["basis"]["kPoint"]["relative"]),
                    ),
                    folding=sphinx_input_parameter_dict['sphinx']["basis"]["folding"],
                    save_memory=bool(sphinx_input_parameter_dict['sphinx']["basis"]["saveMemory"])
                ),
                paw_hamilton=SphinxPawHamiltonian(
                    number_empty_states=sphinx_input_parameter_dict['sphinx']['PAWHamiltonian']['nEmptyStates'],
                    ekt=sphinx_input_parameter_dict['sphinx']['PAWHamiltonian']['ekt'],
                    methfessel_paxton=bool(sphinx_input_parameter_dict['sphinx']['PAWHamiltonian']['MethfesselPaxton']),
                    xc=sphinx_input_parameter_dict['sphinx']['PAWHamiltonian']['xc'],
                    spin_polarized=sphinx_input_parameter_dict['sphinx']['PAWHamiltonian']['spinPolarized'],
                ),
                initial_guess=SphinxInitialGuess(
                    waves=SphinxWaves(paw_basis=sphinx_input_parameter_dict['sphinx']['initialGuess']["waves"]["pawBasis"], lcao=sphinx_input_parameter_dict['sphinx']['initialGuess']["waves"]["lcao"]),
                    rho=SphinxRho(atomic_orbitals=sphinx_input_parameter_dict['sphinx']['initialGuess']["rho"]["atomicOrbitals"]),
                    no_waves_storage=bool(sphinx_input_parameter_dict['sphinx']['initialGuess']["noWavesStorage"]),
                ),
                main=SphinxMain(
                    ric_qn=SphinxRicQN(
                        max_steps=int(sphinx_input_parameter_dict['sphinx']['main']['ricQN']['maxSteps']),
                        max_step_length=float(sphinx_input_parameter_dict['sphinx']['main']['ricQN']['maxStepLength']),
                        born_oppenheimer=BornOppenheimer(
                            scf_diag=ScfDiag(
                                rho_mixing=float(sphinx_input_parameter_dict['sphinx']['main']['ricQN']['bornOppenheimer']['scfDiag']['rhoMixing']),
                                spin_mixing=float(sphinx_input_parameter_dict['sphinx']['main']['ricQN']['bornOppenheimer']['scfDiag']['spinMixing']),
                                delta_energy=sphinx_input_parameter_dict['sphinx']['main']['ricQN']['bornOppenheimer']['scfDiag']['dEnergy'],
                                max_steps=sphinx_input_parameter_dict['sphinx']['main']['ricQN']['bornOppenheimer']['scfDiag']['maxSteps'],
                                preconditioner=SphinxPreConditioner(
                                    type=sphinx_input_parameter_dict['sphinx']['main']['ricQN']['bornOppenheimer']['scfDiag']['preconditioner']["type"],
                                    scaling=sphinx_input_parameter_dict['sphinx']['main']['ricQN']['bornOppenheimer']['scfDiag']['preconditioner']["scaling"],
                                    spin_scaling=sphinx_input_parameter_dict['sphinx']['main']['ricQN']['bornOppenheimer']['scfDiag']['preconditioner']["spinScaling"],
                                ),
                                block_ccg=sphinx_input_parameter_dict['sphinx']['main']['ricQN']['bornOppenheimer']['scfDiag']['blockCCG'],
                            ),
                        ),
                    ),
                ),
            ),
            encut=float(sphinx_input_parameter_dict["EnCut"]),
            kpointcoords=sphinx_input_parameter_dict["KpointCoords"],
            kpointfolding=sphinx_input_parameter_dict["KpointFolding"],
            empty_states=int(sphinx_input_parameter_dict["EmptyStates"]),
            methfessel_paxton=bool(sphinx_input_parameter_dict["MethfesselPaxton"]),
            sigma=float(sphinx_input_parameter_dict["Sigma"]), 
            xcorr=sphinx_input_parameter_dict["Xcorr"],
            vasppot=bool(sphinx_input_parameter_dict["VaspPot"]),
            e_step=int(sphinx_input_parameter_dict["Estep"]),
            ediff=float(sphinx_input_parameter_dict["Ediff"]),
            write_waves=bool(sphinx_input_parameter_dict["WriteWaves"]),
            kj_xc=bool(sphinx_input_parameter_dict["KJxc"]),
            save_memory=bool(sphinx_input_parameter_dict["SaveMemory"]),
            rho_mixing=float(sphinx_input_parameter_dict["rhoMixing"]),
            spin_mixing=float(sphinx_input_parameter_dict["spinMixing"]),
            rho_residual_scaling=float(sphinx_input_parameter_dict["rhoResidualScaling"]),
            spin_residual_scaling=float(sphinx_input_parameter_dict["spinResidualScaling"]),
            check_overlap=bool(sphinx_input_parameter_dict["CheckOverlap"]),
            threads=bool(sphinx_input_parameter_dict["THREADS"]),
            use_on_the_fly_cg_optimization=bool(sphinx_input_parameter_dict["use_on_the_fly_cg_optimization"]),
            ionic_step=int(sphinx_input_parameter_dict["Istep"]),
        ),
        structure=Structure(
            dimension=job_dict["sx"]["input"]["structure"]['dimension'],
            indices=job_dict["sx"]["input"]["structure"]['indices'],
            info=job_dict["sx"]["input"]["structure"]['info'],
            positions=job_dict["sx"]["input"]["structure"]['positions'],
            species=job_dict["sx"]["input"]["structure"]['species'],
            cell=Cell(
                cell=job_dict["sx"]["input"]["structure"]['cell']["cell"],
                pbc=job_dict["sx"]["input"]["structure"]["cell"]['pbc'],
            ),
            units=Units(
                length=job_dict["sx"]["input"]["structure"]['units']["length"],
                mass=job_dict["sx"]["input"]["structure"]['units']["mass"],
            )
        ),
    ),
    calculation_output=SphinxOutput(
        charge_density=SphinxOutputChargeDensity(total=job_dict["sx"]["output"]['charge_density']["total"]),
        electronic_structure=SphinxElectronicStructure(
            efermi=job_dict["sx"]["output"]['electronic_structure']["efermi"],
            eig_matrix=job_dict["sx"]["output"]['electronic_structure']["eig_matrix"],
            k_points=job_dict["sx"]["output"]['electronic_structure']["k_points"],
            k_weights=job_dict["sx"]["output"]['electronic_structure']["k_weights"],
            occ_matrix=job_dict["sx"]["output"]['electronic_structure']["occ_matrix"],
            dos=SphinxDensityOfStates(
                energies=job_dict["sx"]["output"]['electronic_structure']["dos"]["energies"],
                int_densities=job_dict["sx"]["output"]['electronic_structure']["dos"]["int_densities"],
                tot_densities=job_dict["sx"]["output"]['electronic_structure']["dos"]["tot_densities"],
            ),
        ),
        electrostatic_potential=SphinxElectrostaticPotential(total=job_dict["sx"]["output"]['electrostatic_potential']["total"]),
        generic=GenericOutput(
            cells=output_dict["cells"] * ureg.angstrom,
            energy_pot=output_dict["energy_pot"] * ureg.eV,
            energy_tot=output_dict["energy_tot"] * ureg.eV,
            forces=output_dict["forces"] * ureg.eV / ureg.angstrom,
            indices=output_dict.get("indices", None),
            natoms=output_dict.get("natoms", None),
            positions=output_dict["positions"] * ureg.angstrom,
            pressures=output_dict.get("pressures", None),
            steps=output_dict.get("steps", None),
            temperature=output_dict.get("temperature", None),
            unwrapped_positions=output_dict.get("unwarapped_positions", None),
            velocities=output_dict.get("velocities", None),
            volume=output_dict["volume"] * ureg.angstrom * ureg.angstrom * ureg.angstrom,
            dft=OutputGenericDFT(
                energy_free=output_dict["dft"]['energy_free'] * ureg.eV,
                n_valence=output_dict["dft"]['n_valence'],
                bands_k_weights=output_dict["dft"]['bands_k_weights'],
                kpoints_cartesian=output_dict["dft"]['kpoints_cartesian'],
                bands_e_fermi=output_dict["dft"]['bands_e_fermi'],
                bands_occ=output_dict["dft"]['bands_occ'],
                bands_eigen_values=output_dict["dft"]['bands_eigen_values'],
                scf_convergence=output_dict["dft"]['scf_convergence'],
                scf_energy_int=output_dict["dft"]['scf_energy_int'] * ureg.eV,
                scf_energy_free=output_dict["dft"]['scf_energy_free'] * ureg.eV,
                scf_computation_time=output_dict["dft"]['scf_computation_time'],
                scf_energy_zero=output_dict["dft"]['scf_energy_zero'],
                scf_energy_band=output_dict["dft"]['scf_energy_band'],
                scf_electronic_entropy=output_dict["dft"]['scf_electronic_entropy'],
                scf_residue=output_dict["dft"]['scf_residue'],
                energy_int=output_dict["dft"]['energy_int'],
                computation_time=output_dict["dft"]['computation_time'],
                energy_zero=output_dict["dft"]['energy_zero'] * ureg.eV,
                energy_band=output_dict["dft"]['energy_band'] * ureg.eV,
                electronic_entropy=output_dict["dft"]['electronic_entropy'],
                residue=output_dict["dft"]['residue'],
            ),
        ),
    ),
    job_id=job_dict["sx"]["job_id"],
    status=job_dict["sx"]["status"],
)

sphinx_job

SphinxJob(executable=Executable(version='2.6.1', name='sphinx', operation_system_nt=False, executable=None, mpi=False, accepted_return_codes=[0]), server=Server(user='pyiron', host='jupyter-pyiron-2ddev-2dpyiron-2dhdf5-2dformat-2dfnq51935', run_mode='modal', queue=None, qid=None, cores=1, threads=1, new_h5=True, structure_id=None, run_time=None, memory_limit=None, accept_crash=False), calculation_input=SphinxInput(generic_dict=GenericDict(restart_file_list=[], restart_file_dict={}, exclude_nodes_hdf=[], exclude_groups_hdf=[]), interactive=Interactive(interactive_flush_frequency=1, interactive_write_frequency=1), generic=GenericInput(calc_mode='minimize', structure='atoms', fix_symmetry='True', k_mesh_spacing=None, k_mesh_center_shift=None, reduce_kpoint_symmetry=True, restart_for_band_structure=False, path_name=None, n_path=None, fix_spin_constraint=False, max_iter=None, temperature=None, n_ionic_steps=None, n_print=None, temperature_damping_timescale=None, pressure_damping_timescale=N

# Get SPHInX schema

In [24]:
file_name = job.project_hdf5.file_name
file_name

'/home/jovyan/test/sx.h5'

In [25]:
node_lst = list_hdf(file_name=file_name, h5_path="/", recursive=True)[0]
node_lst

['/sx/DICT_VERSION',
 '/sx/HDF_VERSION',
 '/sx/NAME',
 '/sx/OBJECT',
 '/sx/TYPE',
 '/sx/VERSION',
 '/sx/executable',
 '/sx/job_id',
 '/sx/server',
 '/sx/status',
 '/sx/input/generic_dict',
 '/sx/input/interactive',
 '/sx/input/generic/DICT_VERSION',
 '/sx/input/generic/NAME',
 '/sx/input/generic/OBJECT',
 '/sx/input/generic/TYPE',
 '/sx/input/generic/VERSION',
 '/sx/input/generic/data_dict',
 '/sx/input/parameters/CheckOverlap__index_18',
 '/sx/input/parameters/Ediff__index_10',
 '/sx/input/parameters/EmptyStates__index_4',
 '/sx/input/parameters/EnCut__index_1',
 '/sx/input/parameters/Estep__index_9',
 '/sx/input/parameters/HDF_VERSION',
 '/sx/input/parameters/Istep__index_21',
 '/sx/input/parameters/KJxc__index_12',
 '/sx/input/parameters/KpointCoords__index_2',
 '/sx/input/parameters/KpointFolding__index_3',
 '/sx/input/parameters/MethfesselPaxton__index_5',
 '/sx/input/parameters/NAME',
 '/sx/input/parameters/OBJECT',
 '/sx/input/parameters/READ_ONLY',
 '/sx/input/parameters/SaveMe

# Write SPHInX job

In [26]:
node_dict = {node: get_node_from_job_dict(job_dict=job_dict, node=node[1:]) for node in node_lst}

In [27]:
os.remove(file_name)

In [28]:
write_dict_to_hdf(file_name=file_name, data_dict=node_dict)

# Reload job

In [29]:
job = pr.load(job.job_name)

/srv/conda/envs/notebook/lib/python3.10/site-packages/pyiron_base/interfaces/lockable.py:59: LockedWarning: __setitem__ called on <class 'pyiron_atomistics.sphinx.input_writer.Group'>, but object is locked!
  warnings.warn(


In [30]:
job.input

Group({'sphinx': Group({'pawPot': Group({'species': Group([Group({'name': '"Al"', 'potType': '"AtomPAW"', 'element': '"Al"', 'potential': '"Al_GGA.atomicdata"'})])}), 'structure': Group({'cell': array([[7.6533908, 0.       , 0.       ],
       [0.       , 7.6533908, 0.       ],
       [0.       , 0.       , 7.6533908]]), 'species': Group([Group({'element': '"Al"', 'atom': Group([Group({'label': '"spin_0.0"', 'coords': array([0., 0., 0.]), 'movable': 1}), Group({'label': '"spin_0.0"', 'coords': array([0.       , 3.8266954, 3.8266954]), 'movable': 1}), Group({'label': '"spin_0.0"', 'coords': array([3.8266954, 0.       , 3.8266954]), 'movable': 1}), Group({'label': '"spin_0.0"', 'coords': array([3.8266954, 3.8266954, 0.       ]), 'movable': 1})])})])}), 'basis': Group({'eCut': 24.989539079445393, 'kPoint': Group({'coords': array([0.5, 0.5, 0.5]), 'weight': 1, 'relative': 1}), 'folding': array([4, 4, 4]), 'saveMemory': 1}), 'PAWHamiltonian': Group({'nEmptyStates': 7, 'ekt': 0.2, 'MethfesselPaxton': 1, 'xc': 'PBE', 'spinPolarized': 0}), 'initialGuess': Group({'waves': Group({'pawBasis': 1, 'lcao': Group([])}), 'rho': Group({'atomicOrbitals': 1}), 'noWavesStorage': 0}), 'main': Group({'ricQN': Group({'maxSteps': '100', 'maxStepLength': '0.18897261246257704', 'bornOppenheimer': Group({'scfDiag': Group({'rhoMixing': '1.0', 'spinMixing': '1.0', 'dEnergy': 3.674932217565499e-06, 'maxSteps': '100', 'preconditioner': Group({'type': 'KERKER', 'scaling': 1.0, 'spinScaling': 1.0}), 'blockCCG': Group([])})})})})}), 'EnCut': 340, 'KpointCoords': [0.5, 0.5, 0.5], 'KpointFolding': [4, 4, 4], 'EmptyStates': 7, 'MethfesselPaxton': 1, 'Sigma': 0.2, 'Xcorr': 'PBE', 'VaspPot': 0, 'Estep': 100, 'Ediff': 0.0001, 'WriteWaves': 1, 'KJxc': 0, 'SaveMemory': 1, 'rhoMixing': 1.0, 'spinMixing': 1.0, 'rhoResidualScaling': 1.0, 'spinResidualScaling': 1.0, 'CheckOverlap': 1, 'THREADS': 1, 'use_on_the_fly_cg_optimization': 1, 'Istep': 100})

In [31]:
job.output.generic

DataContainer({'dft': DataContainer({'energy_free': array([-228.78316018, -228.78315944]), 'n_valence': {'Al': 3}, 'bands_k_weights': array([0.125, 0.375, 0.375, 0.125]), 'kpoints_cartesian': array([[0.1592067, 0.1592067, 0.1592067],
       [0.1592067, 0.1592067, 0.4776201],
       [0.1592067, 0.4776201, 0.4776201],
       [0.4776201, 0.4776201, 0.4776201]]), 'bands_e_fermi': array([5.864736, 5.702807, 5.702836, 5.702855]), 'bands_occ': array([[[ 2.    ,  2.    ,  2.    ,  2.    , -0.0703, -0.0704, -0.0704,
          0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ],
        [ 2.    ,  2.    ,  2.    ,  2.    ,  2.    ,  2.    , -0.    ,
         -0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ],
        [ 2.    ,  2.    ,  2.    ,  2.    ,  2.    ,  1.0356,  1.0347,
          0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ],
        [ 2.    ,  2.    ,  2.    ,  2.    ,  2.    ,  2.    ,  2.    ,
          2.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ]],

       [[ 2.    ,  2.    ,  2.    ,  2.    , -0.0704, -0.0704, -0.0704,
          0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ],
        [ 2.    ,  2.    ,  2.    ,  2.    ,  2.    ,  2.    , -0.    ,
         -0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ],
        [ 2.    ,  2.    ,  2.    ,  2.    ,  2.    ,  1.0352,  1.0352,
          0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ],
        [ 2.    ,  2.    ,  2.    ,  2.    ,  2.    ,  2.    ,  2.    ,
          2.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ]]]), 'bands_eigen_values': array([[[-5.14  ,  1.4238,  1.4239,  1.4239,  5.9394,  5.9395,  5.9395,
          7.7794,  7.7795,  7.7795, 11.4633, 11.4633, 11.4635],
        [-4.0193, -1.8032,  2.4931,  2.4931,  4.6313,  4.6314,  7.0212,
          7.0213,  8.8035,  9.1646,  9.1649, 10.894 , 11.1662],
        [-2.9064, -0.7059, -0.7059,  1.4615,  3.5798,  5.6986,  5.6987,
          7.8085,  8.0932, 10.1434, 10.1435, 12.2068, 13.4422],
        [-1.8011,  0.3824,  0.3824,  0.3824,  2.5573,  2.5575,  2.5575,
          4.693 , 13.1792, 13.1793, 13.1793, 14.4332, 14.4334]],

       [[-5.14  ,  1.4239,  1.4239,  1.4239,  5.9395,  5.9395,  5.9395,
          7.7795,  7.7795,  7.7795, 11.4635, 11.4635, 11.4635],
        [-4.0192, -1.8032,  2.4931,  2.4931,  4.6314,  4.6314,  7.0213,
          7.0213,  8.8036,  9.1648,  9.1648, 10.8941, 11.1662],
        [-2.9064, -0.7059, -0.7059,  1.4616,  3.5798,  5.6987,  5.6987,
          7.8086,  8.0932, 10.1435, 10.1435, 12.2069, 13.4422],
        [-1.8011,  0.3824,  0.3824,  0.3824,  2.5575,  2.5575,  2.5575,
          4.6931, 13.1794, 13.1794, 13.1794, 14.4334, 14.4334]]]), 'scf_convergence': [True, True], 'scf_energy_int': array([array([-227.76073135, -228.76487746, -228.75055037, -228.75030395,
              -228.75029236])                                            ,
       array([-228.75029155, -228.75029152])], dtype=object), 'scf_energy_free': array([array([-227.79757673, -228.79771858, -228.78341073, -228.78317077,
              -228.78316018])                                            ,
       array([-228.78315943, -228.78315944])], dtype=object), 'scf_computation_time': array([array([1.282293, 2.56702 , 3.856167, 5.198665, 6.542021]),
       array([7.805731, 9.11019 ])], dtype=object), 'scf_energy_zero': array([array([-227.78836538, -228.7895083 , -228.77519564, -228.77495407,
              -228.77494323])                                            ,
       array([-228.77494246, -228.77494246])], dtype=object), 'scf_energy_band': array([array([15.05566482, 13.62260753, 13.80978426, 13.83579452, 13.84051221]),
       array([13.84081386, 13.84101605])], dtype=object), 'scf_electronic_entropy': array([array([5.01306946, 4.46826186, 4.47087874, 4.47175913, 4.47189425]),
       array([4.47190311, 4.4719075 ])], dtype=object), 'scf_residue': array([array([5.57072e-03, 1.65550e-03, 2.37401e-04, 4.03935e-05, 6.62098e-06]),
       array([1.48271e-06, 1.86194e-07])], dtype=object), 'energy_int': arr